In [1]:
import os
from pathlib import Path
current_directory = Path().resolve()
os.chdir(Path(current_directory).resolve().parent.parent)

import pandas as pd
import matplotlib.pyplot as plt
import scienceplots
import chardet

plt.style.use(['science', 'no-latex', 'notebook', 'grid'])
plt.rcParams['text.usetex'] = False

file_path = current_directory / "r_0.csv"

if not file_path.exists():
    raise Exception(f"File {file_path} does not exist.")

with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
file_encoding = result['encoding']

df = pd.read_csv(file_path, encoding=file_encoding, sep=',')

df.head()

,3,5,10,temperature
0,0.000005,0.000009,0.000007,32.18783
1,0.000006,0.000010,0.000008,34.05931
2,0.000006,0.000010,0.000008,35.93079
3,0.000007,0.000011,0.000009,37.80227
4,0.000007,0.000012,0.000010,39.67375


In [2]:
import numpy as np
import scipy.stats as stats
from scipy.constants import R
from src.core.app_settings import NUC_MODELS_TABLE, NUC_MODELS_LIST

ALPHA_MIN = 0.005 # Минимальная конверсия
ALPHA_MAX = 0.995 # Максимальная конверсия
df["10_conversion"] = df["10"].cumsum() / df["10"].cumsum().max()

df["temperature"] = df["temperature"] + 273.15

def calculate_direct_diff_lhs(da_dT, f_a_val):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.log(da_dT / f_a_val)
        result[~np.isfinite(result)] = np.inf  # Заменить NaN и inf на бесконечность
    return result

def calculate_direct_diff_params(slope, intercept, beta):
    Ea = -slope * R
    A = np.exp(intercept) * int(beta)
    return Ea, A

def process_model(temperature, conversion, model_key, beta):
    da_dT = conversion.diff()
    model_func = NUC_MODELS_TABLE[model_key]["differential_form"]
    f_a_val = model_func(1 - conversion)

    lhs = calculate_direct_diff_lhs(da_dT, f_a_val)

    reverse_temperature = 1 / temperature
    valid_mask = np.isfinite(lhs) & np.isfinite(reverse_temperature)  # Отбираем только валидные данные
    
    if np.sum(valid_mask) < len(temperature) * 0.8:  # Если недостаточно данных для линейной регрессии
        return pd.DataFrame({
            'Model': [model_key],
            'R2_score': [np.nan],
            'Ea': [np.nan],
            'A': [np.nan],
        })
    
    # Линейная регрессия только с валидными значениями
    slope, intercept, r_value, p_value, std_err = stats.linregress(reverse_temperature[valid_mask], lhs[valid_mask])
    Ea, A = calculate_direct_diff_params(slope, intercept, beta)
    
    return pd.DataFrame({
        'Model': [model_key],
        'R2_score': [r_value**2],
        'Ea': [Ea],
        'A': [A],
    })

def trim_conversion(temperature, conversion):
    valid_mask = (conversion >= ALPHA_MIN) & (conversion <= ALPHA_MAX)
    trimmed_conversion = conversion[valid_mask]
    trimmed_temperature = temperature[valid_mask]
    
    return trimmed_temperature, trimmed_conversion

def obtain_direct_diff_coeffs(temperature, conversion, beta):
    direct_diff = pd.DataFrame(columns=['Model', 'R2_score', 'Ea', 'A'])
    trimmed_temperature, trimmed_conversion = trim_conversion(temperature, conversion)
    for i, model in enumerate(NUC_MODELS_LIST):
        temp_df = process_model(trimmed_temperature, trimmed_conversion, NUC_MODELS_LIST[i], beta)
        direct_diff = pd.concat([direct_diff, temp_df], ignore_index=True)

    return direct_diff

beta = 3
direct_diff = obtain_direct_diff_coeffs(df["temperature"], df["10_conversion"], beta)

df_sorted = direct_diff.sort_values(by='R2_score', ascending=False)

df_sorted['R2_score'] = df_sorted['R2_score'].round(4)
df_sorted['Ea'] = df_sorted['Ea'].round()
df_sorted['A'] = df_sorted['A'].apply(lambda x: f"{x:.3e}")

df_sorted

C:\Users\davjd\AppData\Local\Temp\ipykernel_41788\961845995.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  direct_diff = pd.concat([direct_diff, temp_df], ignore_index=True)


,Model,R2_score,Ea,A
11,D4,0.9995,193663.0,4.089e+14
31,G7,0.9988,197392.0,8.541e+15
30,G6,0.9983,-212026.0,1.624e-18
9,D2,0.9982,186858.0,3.953e+14
19,F1/3,0.9975,90312.0,3.059e+06
37,R2,0.9974,96662.0,9.644e+06
29,G5,0.9964,-102780.0,1.854e-09
10,D3,0.9947,206362.0,7.225e+15
32,G8,0.9947,206362.0,3.251e+16
38,R3,0.9888,103011.0,2.703e+07
